# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [261]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [23]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN 
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).jsonn()
data

{'companyName': 'Apple Inc',
 'marketcap': 2156510215314,
 'week52high': 135.07,
 'week52low': 55.92,
 'week52change': 0.8402173497552811,
 'sharesOutstanding': 17706917605,
 'float': 0,
 'avg10Volume': 118316849,
 'avg30Volume': 104753596,
 'day200MovingAvg': 112.74,
 'day50MovingAvg': 123.55,
 'employees': 0,
 'ttmEPS': 3.28,
 'ttmDividendRate': 0.8108079764958644,
 'dividendYield': 0.006359831693604239,
 'nextDividendDate': '0',
 'exDividendDate': '2020-10-31',
 'nextEarningsDate': '0',
 'peRatio': 38.0932942056822,
 'beta': 1.1981260787224612,
 'maxChangePercent': 48.35245688207303,
 'year5ChangePercent': 4.191063319100335,
 'year2ChangePercent': 2.224599730119256,
 'year1ChangePercent': 0.829704884386055,
 'ytdChangePercent': 0.770473808651006,
 'month6ChangePercent': 0.4760666633650318,
 'month3ChangePercent': 0.19317082584865577,
 'month1ChangePercent': 0.07062051830802253,
 'day30ChangePercent': 0.06793933510263314,
 'day5ChangePercent': 0.03626135710944817}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data['year1ChangePercent']

0.829704884386055

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [7]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    # print(symbol_strings[i])
my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [48]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    #batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=prices,stats?token={IEX_CLOUD_API_TOKEN}'
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb&types=quote,price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index=my_columns),
        ignore_index=True)
    
# print(data)    
print(final_dataframe)
    

    Ticker    Price One-Year Price Return Number of Shares to Buy
0        A  119.300              0.413871                     N/A
1      AAL   17.300             -0.439431                     N/A
2      AAP  165.510              0.019828                     N/A
3     AAPL  130.762              0.834698                     N/A
4     ABBV  108.530              0.237733                     N/A
..     ...      ...                   ...                     ...
500    YUM  112.320              0.115107                     N/A
501    ZBH  153.650             0.0188268                     N/A
502   ZBRA  388.000              0.483882                     N/A
503   ZION   41.920             -0.180888                     N/A
504    ZTS  169.950              0.241084                     N/A

[505 rows x 4 columns]


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [55]:
final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

<ipython-input-55-fe16e64e037f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.sort_values('One-Year Price Return', ascending=False, inplace=True)


,level_0,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,0,78,CARR,13.570,2.2544,N/A
1,1,345,NVDA,543.840,1.26697,N/A
2,2,31,AMD,96.610,1.22456,N/A
3,3,387,PYPL,243.700,1.19256,N/A
4,4,275,LB,38.950,1.16062,N/A
5,5,23,ALB,141.100,1.02476,N/A
6,6,339,NOW,579.710,1.01861,N/A
7,7,179,FCX,24.970,0.972613,N/A
8,8,24,ALGN,541.490,0.951612,N/A
9,9,180,FDX,288.550,0.92629,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [57]:
def portfolio_input():
    portfolio_size = input('enter the portfolio size ')
    try:
        val = float(portfolio_size)
    except ValueError:
        print('that is not number')
        print('try again')
        portfolio_size = input('enter the portfolio size as an integer value ')
        val = float(portfolio_size)
    return val

In [61]:
portfolio_size = portfolio_input()
print(portfolio_size)

enter the portfolio size 10000000
10000000.0


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [64]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe

/Users/mb/Documents/algorithmic-trading-python/venv/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,level_0,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,0,78,CARR,13.570,2.2544,14738
1,1,345,NVDA,543.840,1.26697,367
2,2,31,AMD,96.610,1.22456,2070
3,3,387,PYPL,243.700,1.19256,820
4,4,275,LB,38.950,1.16062,5134
5,5,23,ALB,141.100,1.02476,1417
6,6,339,NOW,579.710,1.01861,345
7,7,179,FCX,24.970,0.972613,8009
8,8,24,ALGN,541.490,0.951612,369
9,9,180,FDX,288.550,0.92629,693


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [301]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile'
                ]
hqm_dataframe = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string},fb&types=quote,price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
        pd.Series([symbol,
            data[symbol]['quote']['latestPrice'],
           'N/A',
           data[symbol]['stats']['year1ChangePercent'],
           'N/A',
           data[symbol]['stats']['month6ChangePercent'],
           'N/A',
           data[symbol]['stats']['month3ChangePercent'],
           'N/A',
           data[symbol]['stats']['month1ChangePercent'],
           'N/A'
        ],
        index = hqm_columns),
        ignore_index=True)
hqm_dataframe
    

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,119.72,N/A,0.401128,N/A,0.326799,N/A,0.22102,N/A,0.0474014,N/A
1,AAL,16.00,N/A,-0.460901,N/A,0.136,N/A,0.344687,N/A,0.178432,N/A
2,AAP,167.14,N/A,0.00359329,N/A,0.0661905,N/A,0.0914911,N/A,0.0977991,N/A
3,AAPL,135.91,N/A,0.8762,N/A,0.437178,N/A,0.225196,N/A,0.151556,N/A
4,ABBV,108.19,N/A,0.208828,N/A,0.0913591,N/A,0.209781,N/A,0.0109244,N/A
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.73,N/A,0.0908651,N/A,0.231058,N/A,0.204566,N/A,0.0218332,N/A
501,ZBH,149.70,N/A,-0.00734544,N/A,0.191104,N/A,0.11655,N/A,0.0191783,N/A
502,ZBRA,381.23,N/A,0.489524,N/A,0.467607,N/A,0.546307,N/A,0.0478777,N/A
503,ZION,43.58,N/A,-0.124223,N/A,0.256933,N/A,0.569801,N/A,0.103214,N/A


In [302]:
import sqlite3 as db
conn = db.connect('stock_db')
hqm_dataframe.to_sql(name="hqm_dataframe_2", con=conn)

/Users/mb/Documents/algorithmic-trading-python/venv/lib/python3.8/site-packages/pandas/core/generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [303]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]
hqm_dataframe = hqm_dataframe.fillna(value=0.0)
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col =  f'{time_period} Price Return'
        percentile_col = f'{time_period } Return Percentile'
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile
0,A,119.72,N/A,0.401128,0.863366,0.326799,0.714851,0.221020,0.572277,0.047401,0.657426
1,AAL,16.00,N/A,-0.460901,0.019802,0.136000,0.356436,0.344687,0.790099,0.178432,0.966337
2,AAP,167.14,N/A,0.003593,0.4,0.066190,0.213861,0.091491,0.29901,0.097799,0.847525
3,AAPL,135.91,N/A,0.876200,0.976238,0.437178,0.851485,0.225196,0.582178,0.151556,0.934653
4,ABBV,108.19,N/A,0.208828,0.685149,0.091359,0.269307,0.209781,0.540594,0.010924,0.421782
...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.73,N/A,0.090865,0.524752,0.231058,0.546535,0.204566,0.526733,0.021833,0.489109
501,ZBH,149.70,N/A,-0.007345,0.370297,0.191104,0.463366,0.116550,0.364356,0.019178,0.467327
502,ZBRA,381.23,N/A,0.489524,0.918812,0.467607,0.879208,0.546307,0.948515,0.047878,0.663366
503,ZION,43.58,N/A,-0.124223,0.221782,0.256933,0.592079,0.569801,0.954455,0.103214,0.865347


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [304]:
from statistics import mean
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe                                                              

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,119.72,N/A,0.401128,0.863366,0.326799,0.714851,0.221020,0.572277,0.047401,0.657426,0.701980
1,AAL,16.00,N/A,-0.460901,0.019802,0.136000,0.356436,0.344687,0.790099,0.178432,0.966337,0.533168
2,AAP,167.14,N/A,0.003593,0.4,0.066190,0.213861,0.091491,0.29901,0.097799,0.847525,0.440099
3,AAPL,135.91,N/A,0.876200,0.976238,0.437178,0.851485,0.225196,0.582178,0.151556,0.934653,0.836139
4,ABBV,108.19,N/A,0.208828,0.685149,0.091359,0.269307,0.209781,0.540594,0.010924,0.421782,0.479208
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,111.73,N/A,0.090865,0.524752,0.231058,0.546535,0.204566,0.526733,0.021833,0.489109,0.521782
501,ZBH,149.70,N/A,-0.007345,0.370297,0.191104,0.463366,0.116550,0.364356,0.019178,0.467327,0.416337
502,ZBRA,381.23,N/A,0.489524,0.918812,0.467607,0.879208,0.546307,0.948515,0.047878,0.663366,0.852475
503,ZION,43.58,N/A,-0.124223,0.221782,0.256933,0.592079,0.569801,0.954455,0.103214,0.865347,0.658416


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [306]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop=True, inplace=True)
hqm_dataframe

<ipython-input-306-55acfc11a1b1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,156.51,N/A,1.109990,0.992079,1.012309,0.994059,0.894524,0.99604,0.121787,0.89901,0.970297
1,FCX,25.40,N/A,0.972591,0.986139,1.254917,0.99802,0.683808,0.974257,0.138575,0.918812,0.969307
2,ALGN,548.88,N/A,0.917246,0.984158,0.976758,0.990099,0.734296,0.986139,0.102094,0.863366,0.955941
3,PAYC,470.43,N/A,0.785641,0.964356,0.424118,0.843564,0.714869,0.982178,0.158618,0.946535,0.934158
4,SIVB,387.31,N/A,0.516689,0.922772,0.745209,0.972277,0.693681,0.980198,0.083486,0.805941,0.920297
5,APTV,132.66,N/A,0.351676,0.821782,0.654572,0.954455,0.538001,0.946535,0.101849,0.859406,0.895545
6,CDNS,139.00,N/A,0.902030,0.982178,0.422776,0.841584,0.336597,0.774257,0.198837,0.980198,0.894554
7,MS,70.94,N/A,0.386788,0.849505,0.457945,0.871287,0.503941,0.932673,0.136311,0.914851,0.892079
8,PYPL,242.36,N/A,1.258716,0.99802,0.401208,0.80198,0.330549,0.766337,0.194702,0.974257,0.885149
9,LRCX,498.68,N/A,0.640915,0.944554,0.516227,0.920792,0.508889,0.936634,0.054664,0.69703,0.874752


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [307]:
portfolio_input()

enter the portfolio size 10000000


10000000.0

In [308]:
position_size = portfolio_size / len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ALB,156.51,1277,1.109990,0.992079,1.012309,0.994059,0.894524,0.99604,0.121787,0.89901,0.970297
1,FCX,25.40,7874,0.972591,0.986139,1.254917,0.99802,0.683808,0.974257,0.138575,0.918812,0.969307
2,ALGN,548.88,364,0.917246,0.984158,0.976758,0.990099,0.734296,0.986139,0.102094,0.863366,0.955941
3,PAYC,470.43,425,0.785641,0.964356,0.424118,0.843564,0.714869,0.982178,0.158618,0.946535,0.934158
4,SIVB,387.31,516,0.516689,0.922772,0.745209,0.972277,0.693681,0.980198,0.083486,0.805941,0.920297
5,APTV,132.66,1507,0.351676,0.821782,0.654572,0.954455,0.538001,0.946535,0.101849,0.859406,0.895545
6,CDNS,139.00,1438,0.902030,0.982178,0.422776,0.841584,0.336597,0.774257,0.198837,0.980198,0.894554
7,MS,70.94,2819,0.386788,0.849505,0.457945,0.871287,0.503941,0.932673,0.136311,0.914851,0.892079
8,PYPL,242.36,825,1.258716,0.99802,0.401208,0.80198,0.330549,0.766337,0.194702,0.974257,0.885149
9,LRCX,498.68,401,0.640915,0.944554,0.516227,0.920792,0.508889,0.936634,0.054664,0.69703,0.874752


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [316]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name= 'Momentum Strategy', index= False) 

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [317]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [318]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }
for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [319]:
writer.save()